# Part VIII: Asynchronous Programming

## Chapter 18: WebSockets

While HTTP request-response cycles work well for most API interactions, modern applications require real-time, bidirectional communication. WebSockets provide a persistent, full-duplex connection between client and server over a single TCP connection, enabling instant data transfer without polling. FastAPI's native WebSocket support makes it straightforward to implement chat applications, live notifications, collaborative editing, and real-time dashboards.

---

### 18.1 WebSocket Basics: Establishing Persistent Connections

WebSockets begin with an HTTP handshake that upgrades the connection to the WebSocket protocol (`ws://` or `wss://`). Once established, the connection remains open, allowing both parties to send messages at any time.

#### WebSocket Protocol Flow

```
┌─────────────────────────────────────────────────────────────────┐
│              WebSocket Connection Lifecycle                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Client                                                          │
│    │                                                             │
│    │  1. HTTP Upgrade Request                                     │
│    │     GET /ws HTTP/1.1                                        │
│    │     Host: server.com                                       │
│    │     Upgrade: websocket                                     │
│    │     Connection: Upgrade                                    │
│    │     Sec-WebSocket-Key: dGhlIHNhbXBsZSBub25jZQ==            │
│    │──────────────────────────────────────────▶│                  │
│    │                                             │ Server         │
│    │  2. HTTP 101 Switching Protocols          │                  │
│    │     HTTP/1.1 101 Switching Protocols      │                  │
│    │     Upgrade: websocket                     │                  │
│    │     Connection: Upgrade                    │                  │
│    │     Sec-WebSocket-Accept: s3pPLMBiTxaQ...  │                  │
│    │◀─────────────────────────────────────────────│                  │
│    │                                             │                  │
│    │  ═══════════════════════════════════════   │                  │
│    │  WebSocket Connection Established (ws://)  │                  │
│    │  ═══════════════════════════════════════     │                  │
│    │                                             │                  │
│    │  3. Bidirectional Message Flow              │                  │
│    │  ─────────────────────────────────────    │                  │
│    │  Client ──▶ Server: {"msg": "hello"}       │                  │
│    │  Server ──▶ Client: {"reply": "hi"}         │                  │
│    │  Client ──▶ Server: {"msg": "how are you?"} │                  │
│    │  Server ──▶ Client: {"status": "good"}     │                  │
│    │  ─────────────────────────────────────      │                  │
│    │                                             │                  │
│    │  4. Close Connection                        │                  │
│    │     Client or Server sends Close frame      │                  │
│    │◀════════════════════════════════════════════▶│                  │
│                                                                  │
│  Key Differences from HTTP:                                    │
│  • Persistent connection (no repeated handshakes)               │
│  • Full-duplex (both sides send simultaneously)                  │
│  • Lower latency (no headers per message)                       │
│  • Binary and text message support                               │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Basic WebSocket Implementation

```python
# websocket_basic.py - WebSocket fundamentals
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, status
from fastapi.responses import HTMLResponse
import json
import logging

logger = logging.getLogger(__name__)
app = FastAPI()

# Simple HTML client for testing
HTML_CLIENT = """
<!DOCTYPE html>
<html>
<head>
    <title>WebSocket Test</title>
</head>
<body>
    <h1>WebSocket Test Client</h1>
    <div id="status">Disconnected</div>
    <input type="text" id="messageInput" placeholder="Type a message...">
    <button onclick="sendMessage()">Send</button>
    <div id="messages"></div>
    
    <script>
        const ws = new WebSocket("ws://localhost:8000/ws");
        
        ws.onopen = function() {
            document.getElementById("status").innerHTML = "Connected";
            console.log("WebSocket connected");
        };
        
        ws.onmessage = function(event) {
            const msg = document.createElement("div");
            msg.textContent = "Received: " + event.data;
            document.getElementById("messages").appendChild(msg);
        };
        
        ws.onclose = function() {
            document.getElementById("status").innerHTML = "Disconnected";
        };
        
        function sendMessage() {
            const input = document.getElementById("messageInput");
            ws.send(input.value);
            input.value = "";
        }
    </script>
</body>
</html>
"""


@app.get("/")
async def get():
    """Serve test client."""
    return HTMLResponse(HTML_CLIENT)


@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    """
    Basic WebSocket endpoint.
    
    WebSocket lifecycle:
    1. Client connects (HTTP upgrade)
    2. FastAPI calls accept() to confirm
    3. Loop: receive/send messages
    4. Handle disconnect
    """
    # Accept the WebSocket connection
    # Must be called before sending/receiving
    await websocket.accept()
    logger.info("WebSocket connection accepted")
    
    try:
        # Keep connection alive and handle messages
        while True:
            # Receive message from client
            # This is blocking (waits for message) but non-blocking for event loop
            data = await websocket.receive_text()
            logger.info(f"Received: {data}")
            
            # Echo back with modification
            await websocket.send_text(f"Echo: {data}")
            
    except WebSocketDisconnect:
        logger.info("Client disconnected")
    except Exception as e:
        logger.error(f"WebSocket error: {e}")
    finally:
        # Cleanup happens automatically, but explicit close is good practice
        logger.info("WebSocket connection closed")


# WebSocket with path parameters
@app.websocket("/ws/{client_id}")
async def websocket_with_params(websocket: WebSocket, client_id: str):
    """
    WebSocket with URL parameters.
    
    Useful for identifying clients or rooms.
    """
    await websocket.accept()
    
    # Send welcome message with client ID
    await websocket.send_json({
        "type": "connection",
        "client_id": client_id,
        "message": f"Welcome, client {client_id}!"
    })
    
    try:
        while True:
            data = await websocket.receive_text()
            
            # Process based on client_id
            response = {
                "client_id": client_id,
                "received": data,
                "timestamp": datetime.utcnow().isoformat()
            }
            
            await websocket.send_json(response)
            
    except WebSocketDisconnect:
        logger.info(f"Client {client_id} disconnected")
```

**Key WebSocket Methods:**

| Method | Purpose | When to Use |
|--------|---------|-------------|
| `await websocket.accept()` | Accept connection | Must call first thing |
| `await websocket.receive_text()` | Receive string | For JSON or text protocols |
| `await websocket.receive_bytes()` | Receive binary | For images, files, protobuf |
| `await websocket.receive_json()` | Receive & parse JSON | Convenience wrapper |
| `await websocket.send_text(str)` | Send string | Text/JSON messages |
| `await websocket.send_bytes(bytes)` | Send binary | File transfers |
| `await websocket.send_json(dict)` | Send JSON | API-style messaging |
| `await websocket.close(code, reason)` | Close connection | Clean shutdown |

---

### 18.2 Handling Messages: Bidirectional Communication

WebSockets enable complex communication patterns. Proper message handling requires parsing, validation, routing, and error handling.

#### Message Protocol Design

```python
# websocket_messaging.py - Structured messaging
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, HTTPException
from pydantic import BaseModel, ValidationError
from typing import Dict, Any, Optional
from enum import Enum
import json

app = FastAPI()

class MessageType(str, Enum):
    """Standard message types."""
    CHAT = "chat"
    SYSTEM = "system"
    TYPING = "typing"
    PING = "ping"
    PONG = "pong"
    ERROR = "error"

class WebSocketMessage(BaseModel):
    """
    Standardized WebSocket message format.
    
    All messages follow this schema for consistency.
    """
    type: MessageType
    payload: Dict[str, Any]
    timestamp: Optional[str] = None
    sender_id: Optional[str] = None

class ChatPayload(BaseModel):
    """Chat message payload."""
    text: str
    room_id: Optional[str] = None

class TypingPayload(BaseModel):
    """Typing indicator payload."""
    is_typing: bool
    room_id: str


@app.websocket("/ws/chat/{user_id}")
async def chat_websocket(websocket: WebSocket, user_id: str):
    """
    Production-ready WebSocket with message validation.
    
    Features:
    - Structured message protocol
    - Validation with Pydantic
    - Error handling
    - Heartbeat/ping-pong
    """
    await websocket.accept()
    
    # Connection state
    connection_info = {
        "user_id": user_id,
        "connected_at": datetime.utcnow(),
        "message_count": 0
    }
    
    await websocket.send_json({
        "type": "system",
        "payload": {
            "message": "Connected to chat server",
            "user_id": user_id
        }
    })
    
    try:
        while True:
            # Receive raw message
            raw_data = await websocket.receive_text()
            
            try:
                # Parse JSON
                data = json.loads(raw_data)
                
                # Validate message structure
                message = WebSocketMessage(**data)
                connection_info["message_count"] += 1
                
                # Route by message type
                if message.type == MessageType.PING:
                    await handle_ping(websocket)
                    
                elif message.type == MessageType.CHAT:
                    await handle_chat_message(
                        websocket, 
                        user_id, 
                        ChatPayload(**message.payload)
                    )
                    
                elif message.type == MessageType.TYPING:
                    await handle_typing_indicator(
                        websocket,
                        user_id,
                        TypingPayload(**message.payload)
                    )
                    
                else:
                    await send_error(websocket, f"Unknown message type: {message.type}")
                    
            except json.JSONDecodeError:
                await send_error(websocket, "Invalid JSON format")
                
            except ValidationError as e:
                await send_error(websocket, f"Validation error: {str(e)}")
                
    except WebSocketDisconnect:
        logger.info(f"User {user_id} disconnected")
        # Cleanup: remove from active connections, notify others, etc.
        
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        await send_error(websocket, "Internal server error")


async def handle_ping(websocket: WebSocket):
    """Handle ping message (keep-alive)."""
    await websocket.send_json({
        "type": "pong",
        "payload": {"timestamp": datetime.utcnow().isoformat()}
    })


async def handle_chat_message(
    websocket: WebSocket, 
    user_id: str, 
    payload: ChatPayload
):
    """Process chat message."""
    # Validate message content
    if not payload.text or len(payload.text) > 1000:
        await send_error(websocket, "Invalid message length")
        return
    
    # Broadcast to room (simplified - see next section for full implementation)
    response = {
        "type": "chat",
        "sender_id": user_id,
        "payload": {
            "text": payload.text,
            "room_id": payload.room_id,
            "timestamp": datetime.utcnow().isoformat()
        }
    }
    
    # In real app, broadcast to all room members
    await websocket.send_json(response)


async def handle_typing_indicator(
    websocket: WebSocket,
    user_id: str,
    payload: TypingPayload
):
    """Handle typing indicator."""
    # Broadcast typing status to room
    await websocket.send_json({
        "type": "typing",
        "sender_id": user_id,
        "payload": {
            "is_typing": payload.is_typing,
            "room_id": payload.room_id
        }
    })


async def send_error(websocket: WebSocket, message: str):
    """Send standardized error message."""
    await websocket.send_json({
        "type": "error",
        "payload": {"message": message}
    })


# Binary message handling (file uploads)
@app.websocket("/ws/file-upload/{user_id}")
async def file_upload_websocket(websocket: WebSocket, user_id: str):
    """
    WebSocket for binary file uploads.
    
    First message: JSON metadata
    Subsequent messages: Binary chunks
    """
    await websocket.accept()
    
    metadata = None
    chunks = []
    
    try:
        while True:
            # Check message type
            message = await websocket.receive()
            
            if "text" in message:
                # JSON metadata
                data = json.loads(message["text"])
                
                if data.get("type") == "metadata":
                    metadata = data
                    await websocket.send_json({
                        "type": "ack",
                        "message": "Metadata received, send chunks"
                    })
                    
                elif data.get("type") == "complete":
                    # Reassembly complete
                    file_data = b"".join(chunks)
                    await save_file(user_id, metadata, file_data)
                    await websocket.send_json({
                        "type": "success",
                        "message": f"File {metadata['filename']} uploaded"
                    })
                    chunks = []  # Reset
                    
            elif "bytes" in message:
                # Binary chunk
                chunks.append(message["bytes"])
                await websocket.send_json({
                    "type": "chunk_received",
                    "chunk_number": len(chunks)
                })
                
    except WebSocketDisconnect:
        logger.info(f"Upload cancelled by {user_id}")
        # Cleanup partial upload
```

---

### 18.3 Broadcasting: Connection Managers and Pub/Sub

Single WebSocket connections are useful, but real applications need to broadcast messages to multiple clients. Connection managers track active connections and distribute messages.

#### Connection Manager Implementation

```python
# connection_manager.py - Broadcasting to multiple clients
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from typing import List, Dict, Set
import asyncio
import json

app = FastAPI()

class ConnectionManager:
    """
    Manages WebSocket connections for broadcasting.
    
    Features:
    - Track active connections
    - Broadcast to all or specific users
    - Room/channel support
    - User presence tracking
    """
    
    def __init__(self):
        # Active connections: user_id -> WebSocket
        self.active_connections: Dict[str, WebSocket] = {}
        
        # Room memberships: room_id -> Set[user_id]
        self.rooms: Dict[str, Set[str]] = {}
        
        # User metadata: user_id -> metadata
        self.user_metadata: Dict[str, dict] = {}
    
    async def connect(
        self, 
        websocket: WebSocket, 
        user_id: str,
        metadata: dict = None
    ):
        """
        Accept connection and register user.
        
        Args:
            websocket: The WebSocket object
            user_id: Unique identifier for the user
            metadata: Optional user info (name, avatar, etc.)
        """
        await websocket.accept()
        
        # Close existing connection if any (prevent duplicates)
        if user_id in self.active_connections:
            old_ws = self.active_connections[user_id]
            try:
                await old_ws.close()
            except:
                pass
        
        self.active_connections[user_id] = websocket
        self.user_metadata[user_id] = metadata or {}
        
        logger.info(f"User {user_id} connected. Total: {len(self.active_connections)}")
    
    def disconnect(self, user_id: str):
        """Remove connection."""
        if user_id in self.active_connections:
            del self.active_connections[user_id]
        
        # Remove from all rooms
        for room_id, members in self.rooms.items():
            members.discard(user_id)
        
        if user_id in self.user_metadata:
            del self.user_metadata[user_id]
        
        logger.info(f"User {user_id} disconnected. Total: {len(self.active_connections)}")
    
    async def send_personal_message(self, message: dict, user_id: str):
        """Send message to specific user."""
        if user_id in self.active_connections:
            await self.active_connections[user_id].send_json(message)
    
    async def broadcast(self, message: dict, exclude: List[str] = None):
        """
        Broadcast to all connected clients.
        
        Args:
            message: Message dict to send
            exclude: List of user_ids to exclude (e.g., sender)
        """
        exclude = exclude or []
        disconnected = []
        
        for user_id, connection in self.active_connections.items():
            if user_id in exclude:
                continue
            
            try:
                await connection.send_json(message)
            except Exception as e:
                # Mark for cleanup if send fails
                logger.error(f"Failed to send to {user_id}: {e}")
                disconnected.append(user_id)
        
        # Cleanup dead connections
        for user_id in disconnected:
            self.disconnect(user_id)
    
    # Room/Channel management
    async def join_room(self, user_id: str, room_id: str):
        """Add user to room."""
        if room_id not in self.rooms:
            self.rooms[room_id] = set()
        
        self.rooms[room_id].add(user_id)
        
        # Notify room members
        await self.broadcast_to_room(room_id, {
            "type": "system",
            "payload": {
                "event": "user_joined",
                "user_id": user_id,
                "room_id": room_id
            }
        }, exclude=[user_id])
    
    async def leave_room(self, user_id: str, room_id: str):
        """Remove user from room."""
        if room_id in self.rooms:
            self.rooms[room_id].discard(user_id)
            
            await self.broadcast_to_room(room_id, {
                "type": "system",
                "payload": {
                    "event": "user_left",
                    "user_id": user_id,
                    "room_id": room_id
                }
            })
    
    async def broadcast_to_room(
        self, 
        room_id: str, 
        message: dict,
        exclude: List[str] = None
    ):
        """
        Broadcast to all members of a room.
        
        More efficient than iterating all connections.
        """
        if room_id not in self.rooms:
            return
        
        exclude = exclude or []
        disconnected = []
        
        for user_id in self.rooms[room_id]:
            if user_id in exclude:
                continue
            
            if user_id in self.active_connections:
                try:
                    await self.active_connections[user_id].send_json(message)
                except Exception:
                    disconnected.append(user_id)
        
        # Cleanup
        for user_id in disconnected:
            self.disconnect(user_id)
    
    def get_room_members(self, room_id: str) -> List[str]:
        """Get list of users in room."""
        return list(self.rooms.get(room_id, []))
    
    def get_connection_count(self) -> int:
        """Total active connections."""
        return len(self.active_connections)


# Global instance
manager = ConnectionManager()


@app.websocket("/ws/{user_id}")
async def websocket_broadcast(websocket: WebSocket, user_id: str):
    """
    WebSocket with broadcasting support.
    
    Supports:
    - Direct messages
    - Room-based messaging
    - Global broadcasts
    """
    await manager.connect(websocket, user_id, {"joined_at": datetime.utcnow().isoformat()})
    
    try:
        while True:
            data = await websocket.receive_text()
            message = json.loads(data)
            
            msg_type = message.get("type")
            payload = message.get("payload", {})
            
            if msg_type == "join_room":
                room_id = payload.get("room_id")
                await manager.join_room(user_id, room_id)
                await manager.send_personal_message({
                    "type": "system",
                    "payload": {"message": f"Joined room {room_id}"}
                }, user_id)
                
            elif msg_type == "leave_room":
                room_id = payload.get("room_id")
                await manager.leave_room(user_id, room_id)
                
            elif msg_type == "room_message":
                room_id = payload.get("room_id")
                await manager.broadcast_to_room(room_id, {
                    "type": "chat",
                    "sender_id": user_id,
                    "payload": payload
                }, exclude=[user_id])
                
            elif msg_type == "broadcast":
                await manager.broadcast({
                    "type": "broadcast",
                    "sender_id": user_id,
                    "payload": payload
                }, exclude=[user_id])
                
            elif msg_type == "direct_message":
                target_user = payload.get("target_user")
                await manager.send_personal_message({
                    "type": "direct_message",
                    "sender_id": user_id,
                    "payload": payload
                }, target_user)
                
    except WebSocketDisconnect:
        manager.disconnect(user_id)
        
        # Notify rooms that user left
        for room_id in list(manager.rooms.keys()):
            if user_id in manager.rooms[room_id]:
                await manager.broadcast_to_room(room_id, {
                    "type": "system",
                    "payload": {"event": "user_disconnected", "user_id": user_id}
                })
```

#### Redis Pub/Sub for Multi-Server Scaling

```python
# redis_broadcast.py - Scaling across multiple servers
from redis.asyncio import Redis
import asyncio
import json

class RedisConnectionManager(ConnectionManager):
    """
    Extended manager with Redis Pub/Sub for multi-server support.
    
    When running multiple FastAPI instances (horizontal scaling),
    connections are split across servers. Redis Pub/Sub allows
    broadcasting across all servers.
    """
    
    def __init__(self, redis_url: str = "redis://localhost"):
        super().__init__()
        self.redis = Redis.from_url(redis_url, decode_responses=True)
        self.pubsub = self.redis.pubsub()
        self.channel_prefix = "websocket:"
    
    async def connect(self, websocket: WebSocket, user_id: str, metadata: dict = None):
        await super().connect(websocket, user_id, metadata)
        
        # Subscribe to user's personal channel
        await self.pubsub.subscribe(f"{self.channel_prefix}user:{user_id}")
        
        # Start listening for Redis messages
        asyncio.create_task(self._listen_redis(user_id))
    
    async def _listen_redis(self, user_id: str):
        """Listen for messages from Redis and forward to WebSocket."""
        async for message in self.pubsub.listen():
            if message["type"] == "message":
                data = json.loads(message["data"])
                # Forward to WebSocket if user still connected
                if user_id in self.active_connections:
                    await self.active_connections[user_id].send_json(data)
    
    async def broadcast(self, message: dict, exclude: List[str] = None):
        """
        Broadcast via Redis for cross-server communication.
        
        1. Publish to Redis channel
        2. All servers receive and forward to their local connections
        """
        # Local broadcast
        await super().broadcast(message, exclude)
        
        # Publish to Redis for other servers
        await self.redis.publish(
            f"{self.channel_prefix}broadcast",
            json.dumps({"message": message, "exclude": exclude or []})
        )
    
    async def broadcast_to_room(self, room_id: str, message: dict, exclude: List[str] = None):
        """Room broadcast via Redis."""
        # Local
        await super().broadcast_to_room(room_id, message, exclude)
        
        # Redis
        await self.redis.publish(
            f"{self.channel_prefix}room:{room_id}",
            json.dumps({"message": message, "exclude": exclude or []})
        )
    
    async def send_to_user(self, user_id: str, message: dict):
        """Send via Redis (in case user is on different server)."""
        await self.redis.publish(
            f"{self.channel_prefix}user:{user_id}",
            json.dumps(message)
        )
```

---

### 18.4 WebSockets vs SSE: Choosing the Right Technology

Both WebSockets and Server-Sent Events (SSE) enable server-to-client streaming, but they serve different use cases.

#### Comparison: WebSockets vs SSE

```
┌─────────────────────────────────────────────────────────────────┐
│           WebSockets vs Server-Sent Events (SSE)               │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  WebSockets (ws:// / wss://)                                    │
│  ─────────────────────────────────────────────────────────────  │
│  Protocol: Independent, full-duplex                              │
│  Direction: Bidirectional (client↔server)                       │
│  Use Cases:                                                      │
│   • Chat applications                                            │
│   • Collaborative editing                                        │
│   • Real-time games                                              │
│   • Bidirectional streaming                                      │
│                                                                  │
│  Pros:                                                           │
│   ✓ True bidirectional communication                            │
│   ✓ Binary data support                                         │
│   ✓ Lower overhead after handshake                              │
│   ✓ Cross-domain with CORS                                      │
│                                                                  │
│  Cons:                                                           │
│   ✗ More complex protocol                                       │
│   ✗ Harder to scale (stateful connections)                      │
│   ✗ Firewalls/proxies may block                                 │
│   ✗ No automatic reconnection (must implement)                 │
│                                                                  │
│  ─────────────────────────────────────────────────────────────  │
│                                                                  │
│  Server-Sent Events (EventSource)                               │
│  ─────────────────────────────────────────────────────────────  │
│  Protocol: HTTP-based, unidirectional                           │
│  Direction: Server→Client only                                   │
│  Use Cases:                                                      │
│   • Live notifications                                           │
│   • Real-time dashboards                                         │
│   • Log streaming                                                │
│   • Stock price updates                                          │
│                                                                  │
│  Pros:                                                           │
│   ✓ Simpler protocol (just HTTP)                                │
│   ✓ Automatic reconnection with EventSource API                │
│   ✓ Works through standard HTTP proxies                         │
│   ✓ Built-in event IDs for resume                               │
│   ✓ No WebSocket firewall issues                                  │
│                                                                  │
│  Cons:                                                           │
│   ✗ Unidirectional only (client→server needs separate request)  │
│   ✗ No binary support (text only)                               │
│   ✗ Limited to 6 connections per browser (HTTP/1.1)             │
│   ✗ No native IE support (needs polyfill)                        │
│                                                                  │
│  Decision Tree:                                                  │
│  Need client→server streaming?                                   │
│   YES → WebSockets                                               │
│   NO  → SSE (simpler, more reliable)                            │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### SSE Implementation in FastAPI

```python
# sse_implementation.py - Server-Sent Events
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import asyncio
import json

app = FastAPI()

@app.get("/sse/notifications")
async def notifications_stream():
    """
    Server-Sent Events endpoint.
    
    Returns text/event-stream content type.
    Client uses EventSource API in JavaScript.
    """
    async def event_generator():
        """Generate SSE formatted events."""
        event_id = 0
        
        while True:
            # Check for new notifications (from database, Redis, etc.)
            notification = await get_next_notification()
            
            if notification:
                event_id += 1
                
                # SSE format:
                # id: <id>\n
                # event: <event_type>\n
                # data: <json>\n\n
                
                yield f"id: {event_id}\n"
                yield f"event: notification\n"
                yield f"data: {json.dumps(notification)}\n\n"
            
            # Heartbeat to keep connection alive
            yield ": heartbeat\n\n"
            
            await asyncio.sleep(1)  # Check every second
    
    return StreamingResponse(
        event_generator(),
        media_type="text/event-stream",
        headers={
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
            "X-Accel-Buffering": "no"  # Disable nginx buffering
        }
    )


@app.get("/sse/logs/{job_id}")
async def log_stream(job_id: str):
    """
    Stream logs for a background job.
    
    Client can reconnect and resume from last event ID.
    """
    async def log_generator():
        last_id = 0  # Get from query param for resume
        
        while True:
            logs = await get_logs_since(job_id, last_id)
            
            for log in logs:
                last_id = log["id"]
                yield f"id: {last_id}\nevent: log\ndata: {json.dumps(log)}\n\n"
            
            if await is_job_complete(job_id):
                yield f"event: complete\ndata: {json.dumps({'job_id': job_id})}\n\n"
                break
            
            await asyncio.sleep(0.5)
    
    return StreamingResponse(
        log_generator(),
        media_type="text/event-stream"
    )


# Client-side JavaScript for SSE
"""
const eventSource = new EventSource('/sse/notifications');

// Handle specific event types
eventSource.addEventListener('notification', (event) => {
    const data = JSON.parse(event.data);
    console.log('Notification:', data);
});

eventSource.addEventListener('error', (error) => {
    console.error('SSE error:', error);
    // Auto-reconnects automatically!
});

// Close connection
eventSource.close();
"""
```

#### Hybrid Approach: WebSockets + HTTP

```python
# hybrid_approach.py - Best of both worlds
from fastapi import FastAPI, WebSocket, HTTPException

app = FastAPI()

"""
Hybrid Architecture:
- SSE for server→client broadcasts (notifications, updates)
- HTTP POST for client→server actions (send message, update)
- WebSocket only when bidirectional streaming needed (chat, games)

Benefits:
- SSE handles reconnection automatically
- HTTP endpoints are cacheable, testable
- WebSocket only where truly needed
"""

@app.get("/events")
async def server_events():
    """SSE for notifications."""
    # Implementation from above
    pass

@app.post("/chat/send")
async def send_message_http(message: ChatMessage):
    """
    Send message via HTTP POST.
    
    More reliable than WebSocket for important messages.
    Can return confirmation immediately.
    """
    # Save to database
    saved = await save_message(message)
    
    # Broadcast via Redis/SSE to recipients
    await broadcast_message(saved)
    
    return {"status": "sent", "message_id": saved.id}

@app.websocket("/chat/stream")
async def chat_stream(websocket: WebSocket):
    """
    WebSocket only for real-time typing indicators
    and presence (ephemeral data).
    """
    await websocket.accept()
    
    try:
        while True:
            data = await websocket.receive_json()
            
            if data["type"] == "typing":
                # Broadcast typing status (ephemeral, OK if lost)
                await broadcast_typing_status(data)
                
            elif data["type"] == "presence":
                await update_presence(data)
                
    except WebSocketDisconnect:
        pass
```

---

### Summary

In this chapter, you implemented real-time communication:

1. **WebSocket Basics**: Established persistent connections with HTTP upgrade handshake, using `websocket.accept()`, `receive_text()`, and `send_json()` for bidirectional messaging.

2. **Message Handling**: Created structured protocols with Pydantic validation, implemented message routing by type, handled binary data for file uploads, and added error handling with standardized message formats.

3. **Broadcasting**: Built a `ConnectionManager` to track active connections, implemented room-based messaging, and scaled across servers using Redis Pub/Sub for multi-instance deployments.

4. **WebSockets vs SSE**: Distinguished use cases—WebSockets for true bidirectional needs (chat, games), SSE for unidirectional server pushes (notifications, logs)—and implemented both technologies.

**Production Considerations:**
- Use Redis Pub/Sub for horizontal scaling
- Implement heartbeat/ping-pong for connection health
- Handle reconnection at client level (WebSocket) vs automatic (SSE)
- Limit message size to prevent memory issues
- Authenticate WebSocket connections (token in query param or initial message)
- Monitor connection counts and memory usage

---

### What's Next?

**Chapter 19: Containerization** will cover:
- **Dockerizing FastAPI**: Writing optimized multi-stage Dockerfiles that minimize image size and attack surface, including non-root user execution and layer caching
- **Docker Compose**: Managing multi-container applications with PostgreSQL, Redis, and the FastAPI app with proper networking and volume management
- **Production Images**: Using distroless or slim Python images, handling static files, and configuring Gunicorn with Uvicorn workers for production loads

This next chapter prepares your application for deployment by containerizing all components.